In [9]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import psycopg2
from psycopg2 import sql
import json 

In [48]:
""" FUNCIONES """



# 1. Conexión con BD
def conexionBDPostgresSQL():
    # Se omiten los datos de conexión.
    conexion = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="serRIB99",
        host="localhost",
        port="5432"
    )
    return conexion

# 1.2 Insertar datos Generación
def Generacion_Preprocesado_InsertBD_SQLALCHEMY(dataframeGeneracion):
  # Datos de conexión a POSTGRESQL
  usuario = 'postgres'
  password = 'serRIB99'
  servidor = 'localhost'  
  puerto = '5432'
  basedatos = 'postgres'

  # Crear la URL de conexión
  urlConexion = f'postgresql+psycopg2://{usuario}:{password}@{servidor}:{puerto}/{basedatos}'
  # Motor conexión Postgresql
  engine = create_engine(urlConexion)

  # Tabla y Esquema de BD
  esquemaBD = 'tfm'
  tabla = 'generacion_procesada'

  # Insertar en BBDD desde un Dataframe
  dataframeGeneracion.to_sql(tabla, engine, schema=esquemaBD, if_exists='append', index=False)

# 3. Función para ejecutar cualquier comando SQL
def ejecutarComandoSQL(comandoSQL):
  connection = conexionBDPostgresSQL()
  if connection is None:
      return
  try:
      cursor = connection.cursor()
      cursor.execute(comandoSQL)
      connection.commit()
      print(f"Comando ejecutado con éxito: {comandoSQL[0:50]}")
  except Exception as error:
      print(f"Error al ejecutar el comando: {error}")
  finally:
      if connection:
          cursor.close()
          connection.close()

# 2. # Obtener datos generación desde BBDD y generar Dataframe y CSV
def Preprocesamiento_GeneracionEnergia_ToDataframe():
  # Consultar diferentes tipos de energía
  comandoSQL = """
    SELECT fechageneracion, tipoenergiageneracion, valorgeneracion, unidadmedida
    FROM tfm.generacion g
    WHERE g.fechageneracion BETWEEN '2011-01-01' AND '2024-12-31'
  """
  registrosGeneracion = ejecutarComandoSQLSelect(comandoSQL)

  # Diccionario dónde insertar los datos
  datosGeneracion = {
      'fechaGeneracion': [],
      'Carbon': [],
      'CicloCombinado': [],
      'Cogeneracion': [],
      'Eolica': [],
      'FuelGas': [],
      'Hidroeolica': [],
      'Hidraulica': [],
      'MotorDiesel': [],
      'Nuclear': [],
      'OtrasRenovables': [],
      'SolarFotovoltaica': [],
      'SolarTermica': [],
      'TurbinaGas': [],
      'TurbinaVapor': [],
      'TurbinaBombeo': [],
      'ResiduosNoRenov': [],
      'ResiduosRenov': [],
      'UnidadMedida': [],
      'GeneracionTotal': []
  }

  # Diccionario para Mapear tipo de energía a nombre de columna
  conversionTiposGeneracionEnergia = {
    'Carbón': 'Carbon',
    'Ciclo combinado': 'CicloCombinado',
    'Cogeneración': 'Cogeneracion',
    'Eólica': 'Eolica',
    'Fuel + Gas': 'FuelGas',
    'Hidroeólica': 'Hidroeolica',
    'Hidráulica': 'Hidraulica',
    'Motores diésel': 'MotorDiesel',
    'Nuclear': 'Nuclear',
    'Otras renovables': 'OtrasRenovables',
    'Solar fotovoltaica': 'SolarFotovoltaica',
    'Solar térmica': 'SolarTermica',
    'Turbina de gas': 'TurbinaGas',
    'Turbina de vapor': 'TurbinaVapor',
    'Turbinación bombeo': 'TurbinaBombeo',
    'Residuos renovables': 'ResiduosRenov',
    'Residuos no renovables': 'ResiduosNoRenov',
    'Generación total' : 'GeneracionTotal'
  }

  # Procesar los resultados y agregar los datos al diccionario
  for registro in registrosGeneracion:
    fecha = registro[0]
    tipoGeneracion = registro[1]
    valor_generacion = registro[2]
    unidadMedida= registro[3]
    if fecha not in datosGeneracion['fechaGeneracion']:
        datosGeneracion['fechaGeneracion'].append(fecha)
        for key in datosGeneracion.keys():
            if key != 'fechaGeneracion' and key != 'UnidadMedida':
                datosGeneracion[key].append(0)  # Resto de columnas se inician en 0
            elif key == 'UnidadMedida':
                datosGeneracion[key].append('MWh')

    index = datosGeneracion['fechaGeneracion'].index(fecha)
    columna = conversionTiposGeneracionEnergia.get(tipoGeneracion, None)

    if columna:
        datosGeneracion[columna][index] = valor_generacion

  # Crear dataframe desde "Datos Generación"
  dataframeGeneracion = pd.DataFrame(datosGeneracion)

  # DataFrame a CSV
  dataframeGeneracion.to_csv('GeneracionFormateado.csv', index=False)

  # Insertar en BD
  Generacion_Preprocesado_InsertBD_SQLALCHEMY(dataframeGeneracion)

  
# 4. Función para ejecutar cualquier comando SQL (SELECT)
def ejecutarComandoSQLSelect(comandoSQL):
  connection = conexionBDPostgresSQL()
  resultados = []
  if connection is None:
      return
  try:
      cursor = connection.cursor()
      cursor.execute(comandoSQL)
      # Obtener todos los resultados de la consulta
      resultados = cursor.fetchall()

      # Imprimir los resultados
      """for fila in resultados:
          print(fila)"""
      connection.commit()
      print(f"Comando ejecutado con éxito: {comandoSQL[0:50]}")
  except Exception as error:
      print(f"Error al ejecutar el comando: \n{error}")
  finally:
    if connection:
        cursor.close()
        connection.close()
    return resultados




In [52]:
""" MAIN """

ejecutarComandoSQL("TRUNCATE TABLE tfm.generacion_procesada;")

Preprocesamiento_GeneracionEnergia_ToDataframe()

Comando ejecutado con éxito: TRUNCATE TABLE tfm.generacion_procesada;
Comando ejecutado con éxito: 
    SELECT fechageneracion, tipoenergiageneracion
